# Preparing Python


For the evaluation to work, we need to install

- Python >= 3.6
- Numpy
- Scipy
- Matplotlib
- SoundFile
- RunForrest
- JBOF
- Transplant
- Notebook
- Resampy
- Pandas
- tqdm

The easiest way to set this up is to use the included pipfile, and type

```bash
pipenv install
```

into your terminal.

Additionally, you might need to install the [ZeroMQ](http://zeromq.org/) library, either through your package manager (`{dnf|apt|zypper} install zmq-dev{el}` or similar), or using the [official installer](http://zeromq.org/intro:get-the-software).

All Python code in this notebook will work on Windows, macOS, and Linux. However, the download and unzip shell-scripts require a Unix command line (macOS, Linux, or WSL on Windows). If you are using Windows, you will have to do these steps by hand.

# Getting the Corpora

Before we can run the evaluation, we need to aquire the speech database [PTDB-TUG](https://www.spsc.tugraz.at/tools/ptdb-tug) and noise database [QUT-NOISE](https://research.qut.edu.au/saivt/databases/qut-noise-databases-and-protocols/). 

These databases are available freely under the terms of the [Open Database License](http://opendatacommons.org/licenses/odbl/1.0/) (PTDB-TUG) and the [Creative Commons Attribution-ShareAlike 3.0 License](http://creativecommons.org/licenses/by-sa/3.0/) (QUT-NOISE).

You can either download the files manually from the website, or use the following shell script:

In [ ]:
%%bash
# Download metadata from PTBD-TUG
wget -q http://www2.spsc.tugraz.at/databases/PTDB-TUG/DOCUMENTATION/RECORDING-PROTOCOL.txt
wget -q http://www2.spsc.tugraz.at/databases/PTDB-TUG/DOCUMENTATION/SPEAKER-PROFILES.txt
wget -q http://www2.spsc.tugraz.at/databases/PTDB-TUG/DOCUMENTATION/TIMIT-PROMPTS.txt
# Download SPEECH_DATA_ZIPPED.zip from PTDB-TUG (3.9 Gb)
wget -q http://www2.spsc.tugraz.at/databases/PTDB-TUG/SPEECH_DATA_ZIPPED.zip
# Download qutnoise.zip from QUT-NOISE (26.7 Mb)
wget -q https://data.researchdatafinder.qut.edu.au/dataset/a0eed5af-abd8-441b-b14a-8e064bc3d732/resource/8342a090-89e7-4402-961e-1851da11e1aa/download/qutnoise.zip
# Download qutnoisecafe.zip from QUT-NOISE (1.6 Gb)
wget -q https://data.researchdatafinder.qut.edu.au/dataset/a0eed5af-abd8-441b-b14a-8e064bc3d732/resource/9b0f10ed-e3f5-40e7-b503-73c2943abfb1/download/qutnoisecafe.zip
# Download qutnoisecar.zip from QUT-NOISE (1.7 Gb)
wget -q https://data.researchdatafinder.qut.edu.au/dataset/a0eed5af-abd8-441b-b14a-8e064bc3d732/resource/7412452a-92e9-4612-9d9a-6b00f167dc15/download/qutnoisecar.zip
# Download qutnoisehome.zip from QUT-NOISE (1.4 Gb)
wget -q https://data.researchdatafinder.qut.edu.au/dataset/a0eed5af-abd8-441b-b14a-8e064bc3d732/resource/35cd737a-e6ad-4173-9aee-a1768e864532/download/qutnoisehome.zip
# Download qutnoisereverb.zip from QUT-NOISE (1.4 Gb)
wget -q https://data.researchdatafinder.qut.edu.au/dataset/a0eed5af-abd8-441b-b14a-8e064bc3d732/resource/164d38a5-c08e-4e20-8272-793534eb10c7/download/qutnoisereverb.zip
# Download qutnoisestreet.zip from QUT-NOISE (1.6 Gb)
wget -q https://data.researchdatafinder.qut.edu.au/dataset/a0eed5af-abd8-441b-b14a-8e064bc3d732/resource/10eeceae-9f0c-4556-b33a-dcf35c4f4db9/download/qutnoisestreet.zip

Then, unzip them:

In [ ]:
%%bash
# unzip PTDB-TUG files into PTDB-TUG_original:
unzip -q SPEECH_DATA_ZIPPED.zip
mv SPEECH\ DATA PTDB-TUG_original
mv *.txt PTDB-TUG_original/
# unzip QUT-NOISE files into QUT-NOISE_original:
unzip -q qutnoise.zip
unzip -qo qutnoisecafe.zip
unzip -qo qutnoisecar.zip
unzip -qo qutnoisehome.zip
unzip -qo qutnoisereverb.zip
unzip -qo qutnoisestreet.zip
mv QUT-NOISE QUT-NOISE_original

Next, we need to make these databases easily available within Python. For that, we will file them in [JBOF](https://github.com/bastibe/jbof) datasets. JBOF datasets are simple directory structures that contain audio files and JSON metadata, and are accessible as Python modules. 

We use JBOF datasets instead of, say, HDF files, since HDF files are hard to browse and inspect, sometimes break due to data corruption, do not compress audio data well, and tend to be slow when accessed from many threads in parallel.

To create these two datasets from the unzipped files, run the following two Python scripts:

In [25]:
%run import_ptdb_tug.py
%run import_qut_noise.py

 ref M10 si2232395si2188M06 si1419signal M10 sx422si2111 si2051F04 si1060si2169si923 sx360


Now we don't need the zip files and extracted directories any longer, and can delete them:

In [ ]:
%%bash
rm *.zip
rm -r PTDB-TUG_original
rm -r QUT-NOISE_original

# Running the Experiments

There are two sets of experiments, one with white noise and harmonic tone complexes, and the other with speech recordings from the PTDB-TUG dataset and noise recordings from the QUT-NOISE dataset.

Each synthetic experiment was run 10 times for each combination of

- an SNR in [40, 35, ... -20] dB
- 5 base frequencies between 100 Hz and 200 Hz

Each realistic experiment was run 20 times for each combination of

- an SNR in [40, 35, ... -20] dB
- each noise file in QUT-NOISE, plus white noise

Each of these combinations then used the same random starting time in the noise file and a random speech file from the PTDB-TUG training set for each PDA in [MaPS, PEFAC, RAPT, YIN].

**Warning:** These experiments can take a very long time to calculate. We are using [RunForrest](https://github.com/bastibe/runforrest) to save each experiment to disk, and then execute many of them in parallel until all are done. In the code below, set `nprocesses` (second-to-last line) to the number of CPU cores available on your machine.

We use RunForrest instead of, say, IPyParallel or Dask.distributed, since these experiments call into Matlab Mex files, which is prone to crashing, and has on occasion taken down our computers. Since RunForrest runs each experiment in its own process, most crashes are survivable. And even if the computer crashes, RunForrest cashes all pending/done tasks on disk, and can resume computation after a forced reboot.

To call Matlab code from Python, we use [Transplant](https://github.com/bastibe/transplant). We use Transplant instead of the Matlab Engine for Python, since Transplant starts Matlab in a separate subprocess, and will survive most Matlab crashes. Also, it is considerably faster for non-trivial data exchange.

### PDA implementations

PEFAC and RAPT were downloaded from the [VOICEBOX](http://www.ee.ic.ac.uk/hp/staff/dmb/voicebox/voicebox.html) repository in June 2018, and are licensed under the terms of the GNU GPL license.

YIN was downloaded from Alain de Cheveigné's [website](http://audition.ens.fr/adc/), and is licensed as followed:

> Permission to use, copy, modify, and distribute this software without 
> fee is hereby granted FOR RESEARCH PURPOSES only, provided that this
> copyright notice appears in all copies and in all supporting 
> documentation, and that the software is not redistributed for any 
> fee (except for a nominal shipping charge). 
>
> For any other uses of this software, in original or modified form, 
> including but not limited to consulting, production or distribution
> in whole or in part, specific prior permission must be obtained from CNRS.
> Algorithms implemented by this software may be claimed by patents owned 
> by CNRS, France Telecom, Ircam or others.
>
> The CNRS makes no representations about the suitability of this 
> software for any purpose.  It is provided "as is" without express
> or implied warranty.

MaPS-f₀ itself is licensed under the terms of the GNU Lesser General Public License.

### Synthetic Experiment

The following piece of code will run the synthetic experiment. Depending on the number of threads available on your CPU (configure with `nprocesses`), this will take a few hours.

Pitch tracks and metadata will be saved in a new directory `synthetic experiment`, and are accessible as a `TaskList` object.

In [3]:
import random
import numpy
from itertools import product
from tqdm import tqdm
from runforrest import TaskList, defer
from helper import mix, generate_htc
from pdas import maps, pefac, rapt, yin

duration = 5 # s
samplerate = 48000 # Hz

snrs = range(-20, 45, 5) # dB
basefrequencies = range(100, 200, 20) # Hz
num_trials = range(10)
pdas = [maps, pefac, rapt, yin]

combinations = product(snrs, basefrequencies, pdas, num_trials)

tasklist = TaskList('synthetic experiment')

for snr, basefrequency, pda, _ in tqdm(combinations, smoothing=0, desc='Preparing'):
    true_t = numpy.linspace(0, duration, 100*duration)
    true_f = numpy.ones(true_t.shape) * basefrequency
    htc = defer(generate_htc, duration, true_t, true_f, samplerate)
    noise = defer(numpy.random.randn, htc.shape[0])
    test_signal = defer(mix, htc, noise, snr, samplerate)
    estimate = defer(pda, test_signal, samplerate)
    tasklist.schedule(estimate, metadata=dict(
        true_t=true_t,
        true_f=true_f,
        snr=snr,
        pda=pda.__name__,
        basefrequency=basefrequency))

for item in tqdm(tasklist.run(nprocesses=32, print_errors=True), smoothing=0, desc='Processing'):
    pass # just wait

Preparing: 1040it [00:01, 587.75it/s]
Processing: 100%|██████████| 1040/1040 [05:56<00:00,  2.92it/s]


Now that the pitch tracks have been calculated, we will summarize each experiment as a row of performance metrics in a pandas DataFrame for further analysis:

In [ ]:
from runforrest import TaskList
from pandas import DataFrame
import helper
import numpy

tasklist = TaskList('synthetic experiment', exist_ok=True, pre_clean=False)
results = []
for task in tqdm(tasklist.done_tasks()):
    [est_t, est_f, est_p] = task.returnvalue
    true_t = task.metadata['true_t']
    true_f = task.metadata['true_f']
    est_f[numpy.isnan(est_f)] = 0
    for threshold in [False, True]:
        if threshold and metadata['pda'] in ['pefac', 'maps']:
            est_f[est_p < 0.5] = 0
        results.append(dict(
            gpe=helper.gross_pitch_error(true_t, true_f, est_t, est_f),
            fpe=helper.fine_pitch_error(true_t, true_f, est_t, est_f),
            mf0=numpy.mean(true_f[true_f != 0]),
            tpr=helper.true_positive_rate(true_t, true_f, est_t, est_f),
            fpr=helper.false_positive_rate(true_t, true_f, est_t, est_f),
            fnr=helper.false_negative_rate(true_t, true_f, est_t, est_f),
            threshold=threshold,
            snr=task.metadata['snr'],
            pda=task.metadata['pda'],
            basefrequency=task.metadata['basefrequency']))

table = DataFrame(results)
table.to_msgpack('synthetic.msg')

This table looks like this, and is the basis for the evaluation in the publication:

In [19]:
table.head()

,basefrequency,fnr,fpe,fpr,gpe,mf0,pda,snr,threshold,tpr
0,120,0.022,0.002825,NaN,0.0,120.0,pefac,40,False,0.978
1,120,0.022,0.002825,NaN,0.0,120.0,pefac,40,True,0.978
2,100,0.012,0.005316,NaN,0.0,100.0,maps,10,False,0.988
3,100,0.542,0.004748,NaN,0.0,100.0,maps,10,True,0.458
4,100,0.012,0.015594,NaN,0.0,100.0,maps,35,False,0.988


### Realistic Experiment

Before we can run the realistic experiment, we need to split the speech files in a training set and a test set:

In [29]:
import dill
speeches = list(PTDB_TUG.dataset.all_items())
random.shuffle(speeches)
training_set = speeches[:int(len(speeches)*0.6)]
with open('training_set.pkl', 'wb') as f:
    dill.dump(training_set, f)
test_set = speeches[int(len(speeches)*0.6):]
with open('test_set.pkl', 'wb') as f:
    dill.dump(test_set, f)

The following piece of code will run the realistic experiment. Depending on the number of threads available on your CPU (configure with `nprocesses`), this will take a few hours.

Pitch tracks and metadata will be saved in a new directory `realistic experiment`, and are accessible as a `TaskList` object.

In [5]:
import PTDB_TUG
import QUT_NOISE
import random
import numpy
import dill
from itertools import product
from tqdm import tqdm
from runforrest import TaskList, defer
from helper import create_test_signal
from pdas import maps, pefac, rapt, yin

samplerate = 48000

snrs = range(-20, 45, 5) # dB
num_trials =  20
pdas = [maps, pefac, rapt, yin]
noises = list(QUT_NOISE.dataset.all_items()) + ['white noise']

with open('training_set.pkl', 'rb') as f:
    speeches = dill.load(f)
combinations = product(snrs, noises, pdas)
combinations = [(*p, random.choice(speeches), # speech
                 random.uniform(5*60, 25*60)) # noise_start
                for p in list(combinations)*num_trials]

tasklist = TaskList('realistic experiment')

for snr, noise, pda, speech, noise_start in tqdm(combinations, smoothing=0, desc='Preparing'):
    test_signal = defer(create_test_signal, speech, noise, noise_start, snr)
    estimate = defer(pda, test_signal, samplerate)
    tasklist.schedule(estimate, metadata=dict(
        speech=speech,
        noise=noise,
        noise_start=noise_start,
        snr=snr,
        pda=pda.__name__))

for item in tqdm(tasklist.run(nprocesses=32, print_errors=True), smoothing=0, desc='Processing'):
    pass # just wait

Processing: 100%|██████████| 84/84 [00:47<00:00,  1.75it/s]


Now that the pitch tracks have been calculated, we will summarize each experiment as a row of performance metrics in a pandas DataFrame for further analysis:

In [ ]:
from runforrest import TaskList
from pandas import DataFrame
import helper
import numpy

tasklist = TaskList('realistic experiment', exist_ok=True, pre_clean=False)
results = []
for task in tqdm(tasklist.done_tasks()):
    [est_t, est_f, est_p] = task.returnvalue
    true_t = task.metadata['speech'].pitch['time']
    true_f = task.metadata['speech'].pitch['pitch']
    est_f[numpy.isnan(est_f)] = 0
    noise = task.metadata['noise']
    for threshold in [False, True]:
        if threshold and task.metadata['pda'] in ['pefac', 'maps']:
            est_f[est_p < 0.5] = 0
        results.append(dict(
            gpe=helper.gross_pitch_error(true_t, true_f, est_t, est_f),
            fpe=helper.fine_pitch_error(true_t, true_f, est_t, est_f),
            mf0=numpy.mean(true_f[true_f != 0]),
            tpr=helper.true_positive_rate(true_t, true_f, est_t, est_f),
            fpr=helper.false_positive_rate(true_t, true_f, est_t, est_f),
            fnr=helper.false_negative_rate(true_t, true_f, est_t, est_f),
            threshold=threshold,
            snr=task.metadata['snr'],
            pda=task.metadata['pda'],
            noise_start=task.metadata['noise_start'],
            noise_id=noise.name if not isinstance(noise, str) else noise,
            speech_id=task.metadata['speech'].name))

table = DataFrame(results)
table.to_msgpack('realistic.msg')

This table looks like this, and is the basis for the evaluation in the publication:

In [29]:
table.head()

,fnr,fpe,fpr,gpe,mf0,noise_id,noise_start,pda,snr,speech_id,threshold,tpr
0,0.0,0.02482740709004821,0.9783080260303688,0.2689075630252101,185.91496674355713,CAFE-FOODCOURTB-2,933.306218,pefac,45,F04_si1053,False,1.0
1,0.15126050420168066,0.020635975397267657,0.06724511930585683,0.2079207920792079,185.91496674355713,CAFE-FOODCOURTB-2,933.306218,pefac,45,F04_si1053,True,0.8487394957983193
2,0.0,0.025604634398105244,0.982174688057041,0.41496598639455784,92.98431461359183,STREET-KG-1,1424.727247,pefac,45,M10_si2218,False,1.0
3,0.21768707482993196,0.02511318305704104,0.05525846702317291,0.3652173913043478,92.98431461359183,STREET-KG-1,1424.727247,pefac,45,M10_si2218,True,0.782312925170068
4,0.0,0.03358656881014878,0.9920255183413078,0.012345679012345678,209.3001109935309,STREET-CITY-2,758.023062,maps,45,F02_si780,False,1.0
